In [1]:
from pathlib import Path
import json

import gtfstk as gt
import pandas as pd
import numpy as np


DATA_DIR = Path('../data')
OUT_DIR = Path('../output')

In [2]:
path = DATA_DIR/'wellington_gtfs_20171016.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.assess_quality()

indicator                    value
0     num_route_short_names_duplicated                        0
1    frac_route_short_names_duplicated                        0
2          num_stop_time_dists_missing                   340007
3         frac_stop_time_dists_missing                        1
4            num_direction_ids_missing                        0
5           frac_direction_ids_missing                        0
6             num_trips_missing_shapes                        0
7            frac_trips_missing_shapes                        0
8          num_departure_times_missing                        0
9         frac_departure_times_missing                        0
10   num_first_departure_times_missing                        0
11  frac_first_departure_times_missing                        0
12    num_last_departure_times_missing                        0
13   frac_last_departure_times_missing                        0
14                          assessment  probably a fixable feed

In [8]:
shapes_g = feed.shapes_to_geojson()
path = Path('../wellington_shapes_20171016.geojson')
with path.open('w') as tgt:
    json.dump(shapes_g, tgt)

In [12]:
"""
If no shapes, use stops only.
If shapes, then add distances to stop times and to shapes
"""
feed = feed.append_dist_to_shapes()
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)


In [30]:

def get_trip_points(feed, trip_id, max_sample_points=100):
    """
    """
    # Add stop points to list
    st = feed.stop_times
    st = st[st['trip_id'] == trip_id].copy().merge(feed.stops)
    points = st[['stop_lon', 'stop_lat', 'shape_dist_traveled']].values
    
    n = max_sample_points
    k = st.shape[0]
    
    # Add n - k more points evenly
    assert k < n
    dists = st['shape_dist_traveled'].values
    d = dists[-1]
    delta = d/(n - 1)
    marks = [i*delta for i in range(n)]
    stop_indices = set(np.digitize(dists, marks) - 1)

    print(len(marks))
    print(stop_indices)
    

In [31]:
tid = feed.trips['trip_id'].iat[100]
get_trip_points(feed, tid)

100
{0, 99, 36, 5, 6, 7, 68, 11, 12, 13, 43, 15, 77, 50, 85, 88, 91, 93}


In [24]:
feed.stops

stop_id stop_code                                         stop_name  \
0       7088      7088                     Ira Street at Devonshire Road   
1       1843      1843                  High Street Masterton (near 110)   
2       5496      5496      Thorndon Quay at Davis Street (opposite 125)   
3       1001      1001                      Paraparaumu Station - Stop B   
4       1726      1726                         Otaki College - Mill Road   
5       2162      2162      Warspite Avenue at Castor Crescent (near 50)   
6       8065      8065                      Dowse Drive at Miromiro Road   
7       7238      7238        Darlington Road at Puriri Street (near 26)   
8       8269      8269                  Waiwhetu Road at Durham Crescent   
9       1196      1196                      Rimu Road at KCDC (opposite)   
10      7240      7240                        Darlington Road (near 104)   
11      WELL      WELL                                Wellington Station   
12      7163      7163                      Houghton Bay - The Esplanade   
13      1556      1556                    Tutere Street at Hemara Street   
14      7030      7030                        Caledonia Street (near 10)   
15      9569      9569                  Fergusson Drive at Merton Street   
16      4312      4312                   Tinakori Road at St Mary Street   
17      1610      1610                            Reeves Street (near 7)   
18      5499      5499               Thorndon Quay at Wellington Station   
19      2812      2812                           Te Hiko Street (near 8)   
20      7145      7145  Murchison Street at Domanski Crescent (opposite)   
21      6927      6927        Hornsey Road at Auckland Terrace (near 25)   
22      3927      3927                  Raiha Street at Broken Hill Road   
23      1504      1504                    Parata Street at Sylvan Avenue   
24      1806      1806                 SH53 at Camp Road (hail and ride)   
25      5454      5454                  Bombay Street at Tarikaka Street   
26      9964      9964                  Wellington Road at Dublin Street   
27      9984      9984       Fitzherbert Road at Gospel Chapel (near 24)   
28      6041      6041           Dundas Street at Monro Street (near 55)   
29      9954      9954                           Mohaka Street (near 17)   
...      ...       ...                                               ...   
2890    6762      6762        Todman Street at Mitchell Street (near 97)   
2891    8241      8241             Molesworth Street at Partridge Street   
2892    EPUN      EPUN                                     Epuni Station   
2893    3268      3268              Johnsonville Road at Disraeli Street   
2894    7794      7794                             Ohiro Road (near 457)   
2895    8113      8113                   Queens Drive at Central Library   
2896    8585      8585                   Fergusson Drive (opposite 1055)   
2897    5391      5391                        Campbell Street (near 106)   
2898    9601      9601         Birchville School opposite Gemstone Drive   
2899    2106      2106                        Mungavin Avenue (near 145)   
2900    7135      7135                           Island Bay - The Parade   
2901    3020      3020          Erris Street at Ironside Road (near 101)   
2902    7017      7017                    Wellington Hospital (opposite)   
2903    SOLW      SOLW                                    Solway Station   
2904    5138      5138              Churchill Drive near Chartwell Drive   
2905    3508      3508            Helston Road at Warren Street (near 6)   
2906    NGAI       NaN                                      gaio Station   
2907    TAWA       NaN                                       awa Station   
2908    WOBU       NaN                                     oburn Station   
2909    PETO       NaN                                     etone Station   
2910    PARA       NaN                               